In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Height"
cohort = "GSE152073"

# Input paths
in_trait_dir = "../../input/GEO/Height"
in_cohort_dir = "../../input/GEO/Height/GSE152073"

# Output paths
out_data_file = "../../output/preprocess/Height/GSE152073.csv"
out_gene_data_file = "../../output/preprocess/Height/gene_data/GSE152073.csv"
out_clinical_data_file = "../../output/preprocess/Height/clinical_data/GSE152073.csv"
json_path = "../../output/preprocess/Height/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression data from Brazilian SPAH study"
!Series_summary	"This study is part of previous epidemiologic project, including a population-based survey (Sao Paulo Ageing & Health study (SPAH Study). The data from this study was collected between 2015 to 2016 and involved elderly women (ages ≥65 yeas) living in the Butanta district, Sao Paulo. The purpose of the study was identification of association between transcriptome and the osteo metabolism diseases phenotype, like osteoporosis, vertebral fracture and coronary calcification."
!Series_summary	"Peripheral blood cells suffer alterations in the gene expression pattern in response to perturbations caused by calcium metabolism diseases.   The purpose of this study is to identify possible molecular markers associated with osteoporosis, vertebral fractures and coronary calcification in elderly women from community from Brazilian SPAH study. Vertebral fractures were the most common clinical manife

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For height (our trait variable)
# Looking at sample characteristics dictionary, height is in key 2
# However, there appears to be only one sample with height data - not sufficient for analysis
trait_row = None  # Setting to None since effective data is unavailable

# For age
# Age is available in key 1
age_row = 1  

# For gender
# Looking at the dictionary, all samples are female (key 0)
# Since there's only one value, gender is constant and not useful for our analysis
gender_row = None  

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert height values to continuous numeric format."""
    if pd.isna(value):
        return None
    # Extract numeric value after colon
    try:
        # Extract the height value in cm
        if "height (cm):" in value:
            height_value = float(value.split(':')[1].strip())
            return height_value
        return None
    except:
        return None

def convert_age(value):
    """Convert age values to continuous numeric format."""
    if pd.isna(value):
        return None
    try:
        # Extract the age in years
        if "age (years):" in value:
            age_value = float(value.split(':')[1].strip())
            return age_value
        return None
    except:
        return None

def convert_gender(value):
    """
    Convert gender values to binary format (0 for female, 1 for male).
    Not used in this dataset since gender is constant (all female).
    """
    if pd.isna(value):
        return None
    
    value_lower = value.lower()
    if "female" in value_lower:
        return 0
    elif "male" in value_lower:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We've already loaded clinical_data in a previous step
    clinical_selected = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    preview = preview_df(clinical_selected)
    print("Preview of processed clinical data:")
    print(preview)
    
    # Save the processed clinical data
    clinical_selected.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print(f"Skipping clinical feature extraction as trait data is unavailable.")


Skipping clinical feature extraction as trait data is unavailable.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Matrix table marker not found in first 100 lines


KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'ID' not found in columns."

Trying alternative approach to read the gene data:


Column names: Index(['GSM4602151', 'GSM4602152', 'GSM4602153', 'GSM4602154', 'GSM4602155'], dtype='object')
First 20 row IDs: Index(['TC01000005.hg.1', 'TC01000006.hg.1', 'TC01000007.hg.1',
       'TC01000008.hg.1', 'TC01000009.hg.1', 'TC01000010.hg.1',
       'TC01000011.hg.1', 'TC01000012.hg.1', 'TC01000013.hg.1',
       'TC01000014.hg.1', 'TC01000015.hg.1', 'TC01000016.hg.1',
       'TC01000017.hg.1', 'TC01000018.hg.1', 'TC01000019.hg.1',
       'TC01000020.hg.1', 'TC01000021.hg.1', 'TC01000022.hg.1',
       'TC01000023.hg.1', 'TC01000024.hg.1'],
      dtype='object', name='ID_REF')


### Step 4: Gene Identifier Review

In [5]:
# Based on the identifiers shown in gene expression data (TC01000005.hg.1, TC01000006.hg.1, etc.), 
# these appear to be Affymetrix Transcriptome Analysis Console (TAC) probeset IDs, not standard human gene symbols.
# These identifiers will need to be mapped to human gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE152073
Line 6: !Series_title = Gene expression data from Brazilian SPAH study
Line 7: !Series_geo_accession = GSE152073
Line 8: !Series_status = Public on Jun 10 2020
Line 9: !Series_submission_date = Jun 09 2020
Line 10: !Series_last_update_date = Sep 29 2021
Line 11: !Series_pubmed_id = 32602654
Line 12: !Series_pubmed_id = 34493690
Line 13: !Series_summary = This study is part of previous epidemiologic project, including a population-based survey (Sao Paulo Ageing & Health study (SPAH Study). The data from this study was collected between 2015 to 2016 and involved elderly women (ages ≥65 yeas) living in the Butanta district, Sao Paulo. The purpose of the study was identification of association between


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49, 60, 30, 30, 191], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST00000408384 // MIR1

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Looking at the gene identifiers and annotation data:
# - Gene expression data has probe IDs like 'TC01000005.hg.1' in its index
# - Gene annotation dataframe has the same identifiers in the 'ID' column
# - The gene symbols are in the 'gene_assignment' column, which contains gene information

# 2. Extract the gene mapping data
# The 'ID' column contains the probe identifiers
# The 'gene_assignment' column contains gene information including gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the mapped gene data
print("\nPreview of gene data after mapping:")
print(gene_data.shape)
print(gene_data.index[:10])  # Print first 10 gene symbols



Preview of gene data after mapping:
(49166, 90)
Index(['A-', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1', 'A1-', 'A10', 'A11'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
import numpy as np
import os
import gzip

# 1. Extract gene expression data using the alternative approach that worked in Step 3
with gzip.open(matrix_file, 'rt') as file:
    # Find the start of the data
    for line in file:
        if '!series_matrix_table_begin' in line:
            break
            
    # Read the headers and data
    gene_data = pd.read_csv(file, sep='\t', index_col=0)

# Check if we have gene data before proceeding
if gene_data.empty:
    print("No gene expression data found in the matrix file.")
    is_gene_available = False
else:
    is_gene_available = True
    print(f"Gene data shape before normalization: {gene_data.shape}")

    # Normalize gene symbols using the NCBI Gene database information
    try:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
        
        # Save the normalized gene data to the output file
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        normalized_gene_data.to_csv(out_gene_data_file)
        print(f"Normalized gene data saved to {out_gene_data_file}")
    except Exception as e:
        print(f"Error normalizing gene data: {e}")
        is_gene_available = False
        normalized_gene_data = gene_data  # Use original data if normalization fails

# 2. Link clinical and genetic data
# In Step 2, we determined that Height data is not available in this dataset (trait_row = None)
# Create a minimal clinical dataframe with the trait column (containing NaNs)
if is_gene_available:
    sample_ids = gene_data.columns
    minimal_clinical_df = pd.DataFrame(index=sample_ids)
    minimal_clinical_df[trait] = np.nan  # Add the trait column with NaN values

    # If we have age and gender data from Step 2, add those columns
    if age_row is not None:
        minimal_clinical_df['Age'] = get_feature_data(clinical_data, age_row, 'Age', convert_age).iloc[0]

    if gender_row is not None:
        minimal_clinical_df['Gender'] = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender).iloc[0]

    minimal_clinical_df.index.name = 'Sample'

    # Save this minimal clinical data for reference
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    minimal_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")

    # Create a linked dataset 
    if is_gene_available and normalized_gene_data is not None:
        linked_data = pd.concat([minimal_clinical_df, normalized_gene_data.T], axis=1)
        linked_data.index.name = 'Sample'
        print(f"Linked data shape: {linked_data.shape}")
    else:
        linked_data = minimal_clinical_df
        print("No gene data to link with clinical data.")
else:
    # Create a minimal dataframe with just the trait for the validation step
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No gene data available, creating minimal dataframe for validation.")

# 4 & 5. Validate and save cohort information
# Since trait_row was None in Step 2, we know Height data is not available
is_trait_available = False  # Height data is not available

note = "Dataset contains gene expression data but no Height measurements. This dataset is not usable for studying Height associations."

# For datasets without trait data, we set is_biased to False
# This indicates the dataset is not usable due to missing trait data, not due to bias
is_biased = False

# Final validation
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Since there is no trait data, the dataset is not usable for our association study
# So we should not save it to out_data_file
print(f"Dataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset does not contain Height data and cannot be used for association studies.")

Gene data shape before normalization: (46362, 90)
Gene data shape after normalization: (0, 90)
Normalized gene data saved to ../../output/preprocess/Height/gene_data/GSE152073.csv
Clinical data saved to ../../output/preprocess/Height/clinical_data/GSE152073.csv
Linked data shape: (90, 2)
Abnormality detected in the cohort: GSE152073. Preprocessing failed.
Dataset usability: False
Dataset does not contain Height data and cannot be used for association studies.
